#  1. 데이터 획득 (크롤링)

##### (1) 당일 지역별 코로나 누적확진자수 및 신규확진자수

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd

In [2]:
browser = webdriver.Chrome('./chromedriver.exe')
url = 'https://search.naver.com/search.naver?query=%EC%BD%94%EB%A1%9C%EB%82%98+%ED%98%84%ED%99%A9&ie=utf8&sm=whl_nht'
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [3]:
section = soup.select('tbody > tr')
section = list(section)

In [4]:
data_list = [[sub_section.select('span')[0].text.strip(),
              sub_section.select('span')[1].text.replace(",",""),
              sub_section.select('span.confirmed_case')[0].text
              ] for num, sub_section in enumerate(section)]

data_list

[['서울', '76814', '569'],
 ['경기', '68671', '513'],
 ['대구', '13287', '95'],
 ['인천', '11406', '102'],
 ['부산', '11077', '78'],
 ['경남', '9683', '68'],
 ['경북', '6866', '64'],
 ['충남', '6639', '92'],
 ['검역', '5761', '23'],
 ['대전', '5463', '50'],
 ['강원', '5268', '26'],
 ['충북', '4991', '38'],
 ['울산', '4037', '40'],
 ['광주', '3948', '28'],
 ['전북', '3441', '45'],
 ['제주', '2524', '30'],
 ['전남', '2487', '8'],
 ['세종', '954', '10'],
 ['미국', '38040498', '146,791'],
 ['인도', '32512366', '37,593'],
 ['브라질', '20645537', '30,671'],
 ['러시아', '6709605', '18,972'],
 ['영국', '6601149', '35,584'],
 ['프랑스', '6538328', '20,857'],
 ['터키', '6273651', '19,970'],
 ['아르헨티나', '5155079', '6,994']]

In [5]:
columns = ['지역', '누적확진자', '신규확진자']
df_area = pd.DataFrame(data_list, columns = columns)

df_area.tail()

,지역,누적확진자,신규확진자
21,러시아,6709605,"18,972"
22,영국,6601149,"35,584"
23,프랑스,6538328,"20,857"
24,터키,6273651,"19,970"
25,아르헨티나,5155079,"6,994"


In [7]:
overseas = ['미국', '인도', '브라질', '러시아', '영국',
            '프랑스','터키','아르헨티나']

condition = df_area.지역.isin(overseas) == False
df_area = df_area[condition]
df_area['지역'].unique

df_area

,지역,누적확진자,신규확진자
0,서울,76814,569
1,경기,68671,513
2,대구,13287,95
3,인천,11406,102
4,부산,11077,78
5,경남,9683,68
6,경북,6866,64
7,충남,6639,92
8,검역,5761,23
9,대전,5463,50


##### (2) 최근 7일간 국내발생 및 해외유입 코로나 신규확진자수 

 - 1차:  HTML 구조 파학 --> 실패
 - 2차:  CSS 접근       --> 실패
 - 3차: Listly - Data Scraper 도구 사용
 ---
 
 추가 스터디 예정: CSS언어, HTML, DOM

##### 2차: BeautifulSoup 사용 

##### 3차: Listly - Data Scraper 도구 사용

link: https://chrome.google.com/webstore/detail/listly-data-scraper/ihljmnfgkkmoikgkdkjejbkpdpbmcgeh

#  2. 데이터 저장

##### Excel 저장

##### (1) 당일 지역별 코로나 누적확진자수 및 신규확진자수

In [8]:
df_area.to_excel('./covid19_area.xlsx', index = False)

##### (2) 최근 7일간 국내발생 및 해외유입 코로나 신규확진자수 

In [ ]:
#df_data.to_excel('./covid19_7days.xlsx', index = False)

##### 신규DB생성

In [19]:
import pymysql
import pandas as pd

In [20]:
connect = pymysql.connect(host="127.0.0.1", user="root", password="1234",
                          charset='utf8',cursorclass=pymysql.cursors.DictCursor)
try:
    cur = connect.cursor()                                    
    
    cur.execute("DROP DATABASE IF EXISTS projectDB")
    cur.execute("CREATE DATABASE projectDB")

    cur.execute("SHOW DATABASES")

    db_list = cur.fetchall()

    for db in db_list:
        print(db)
        
except Exception as e:
    print("Exeception occured:{}".format(e))

# finally:
#     connect.close()

{'Database': 'employees'}
{'Database': 'hanbitdb'}
{'Database': 'information_schema'}
{'Database': 'modeldb'}
{'Database': 'mydb'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'projectdb'}
{'Database': 'sakila'}
{'Database': 'shopdb'}
{'Database': 'sqldb'}
{'Database': 'sys'}
{'Database': 'world'}


##### (1) 당일 지역별 코로나 누적확진자수 및 신규확진자수

##### TABLE 생성

In [28]:
connect = pymysql.connect(host="127.0.0.1", user="root", password="1234",
                          charset='utf8',cursorclass=pymysql.cursors.DictCursor)

In [29]:
try:

    cur = connect.cursor() 
    cur.execute("USE projectDB")
    cur.execute("DROP TABLE IF EXISTS covid19")
    cur.execute("CREATE TABLE IF NOT EXISTS covid19(city VARCHAR(10), acc_num INT, new_num INT)")
    cur.execute("SHOW TABLES")

    rows = cur.fetchall()

    for row in rows:
        print(row)

except Exception as e:
    print("Exeception occured:{}".format(e))

# finally:
#     connect.close()

{'Tables_in_projectdb': 'covid19'}


##### Data 입력

In [30]:
covid_status = pd.read_excel('./covid19_area.xlsx',header = 0, usecols = 'A:C')
covid_status.values

array([['서울', 76814, 569],
       ['경기', 68671, 513],
       ['대구', 13287, 95],
       ['인천', 11406, 102],
       ['부산', 11077, 78],
       ['경남', 9683, 68],
       ['경북', 6866, 64],
       ['충남', 6639, 92],
       ['검역', 5761, 23],
       ['대전', 5463, 50],
       ['강원', 5268, 26],
       ['충북', 4991, 38],
       ['울산', 4037, 40],
       ['광주', 3948, 28],
       ['전북', 3441, 45],
       ['제주', 2524, 30],
       ['전남', 2487, 8],
       ['세종', 954, 10]], dtype=object)

In [31]:
try:
    cur = connect.cursor()          
    for row in covid_status.values:
        ins = "INSERT INTO covid19 VALUES ('{}', {}, {})"
        cur.execute("USE projectDB")
        cur.execute("SELECT * FROM covid19")
        cur.execute(ins.format(*row))

        connect.commit()


except Exception as e:
    print("Exeception occured:{}".format(e))


##### (2) 최근 7일간 국내발생 및 해외유입 코로나 신규확진자수 

##### TABLE 생성

In [32]:
try:
    
    cur = connect.cursor() 
    cur.execute("USE projectDB")
    cur.execute("DROP TABLE IF EXISTS covid19_2")
    cur.execute("CREATE TABLE IF NOT EXISTS covid19_2(city VARCHAR(10), acc_num INT, new_num INT)")
    cur.execute("SHOW TABLES")

    rows = cur.fetchall()

    for row in rows:
        print(row)

except Exception as e:
    print("Exeception occured:{}".format(e))
  

{'Tables_in_projectdb': 'covid19'}
{'Tables_in_projectdb': 'covid19_2'}


In [33]:
connect.close()

##### Data 입력

#  3. 데이터분석및시각화

##### (1) 당일 지역별 코로나 누적확진자수 및 신규확진자수

##### Data 불러오기

In [34]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db= 'projectDB', charset='utf8')
cur = conn.cursor()
cur.execute("SELECT * FROM covid19")
data_tuple = cur.fetchall()
df_area = pd.DataFrame(data_tuple, columns = ["지역", "누적확진자", "신규확진자"])
df_area

,지역,누적확진자,신규확진자
0,서울,76814,569
1,경기,68671,513
2,대구,13287,95
3,인천,11406,102
4,부산,11077,78
5,경남,9683,68
6,경북,6866,64
7,충남,6639,92
8,검역,5761,23
9,대전,5463,50


##### 시각화

##### (2) 최근 7일간 국내발생 및 해외유입 코로나 신규확진자수 

##### Data 불러오기

In [ ]:
data1, data2 == "",""
row=None

conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db= 'projectDB', charset='utf8')
cur = conn.cursor()

cur.execute("SELECT * FROM covid19_2")

print("LABEL-1\t\tLABEL-2")

while 1:
    row = cur.fetchone()
    if row == None:
        break
    data1 = row[0]
    data2 = row[1]
    
ex =[]
    ex += [row[0], row[1]
    
    print("{}\t\t{}".format(data1, data2))
print(ex)
    

##### 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import font_manager, rc
import platform
if platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
elif platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
else:
    print('Check your OS system')

In [ ]:
df = pd.read_excel('./covid19_7days.xlsx')
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df['LABEL-1'] = df['LABEL-2']
df.head()

In [ ]:
plt.figure(figsize = (0, 4))